In [168]:
import numpy as np
import matplotlib.pyplot as plt

from termcolor import colored, cprint

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data.dataset import Dataset
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [127]:
data = np.loadtxt('dataWhole.txt', dtype=str, delimiter = '\n')


In [128]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [129]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    return letter, num

In [130]:
def toBoards(game, color, turn):
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, turn + 1):
        turnC = toTurn(game[i])
        if curPlayer == 1:
            blackBoard[turnC] = 1
        else:
            whiteBoard[turnC] = -1
        curPlayer *= -1
    curBoard = np.zeros(shape=(3, 15, 15))
    curBoard[0,:] = playerBoard
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    label = toTurn(game[turn + 1])
    return curBoard, label

In [131]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

In [156]:
def takeRandom():
    game = []
    while len(game) <= 4:
        gameNum = np.random.randint(data.shape[0])
        game = data[gameNum].split()
    color = game[0]
    turn = np.random.randint((len(game) - 2) // 2)
    if color[0] == 'b':
        board, label = toBoards(game, color, turn * 2)
    else:
        board, label = toBoards(game, color, turn * 2 + 1)

    return board, label[0] * 15 + label[1]

In [157]:
def makeBatch(size):
    batch = np.zeros(shape=(size, 3, 15, 15))
    labels = np.zeros(shape=size, dtype=np.long)
    for i in range(size):
        batch[i,:,:,:], labels[i] = takeRandom()
    return torch.from_numpy(batch), torch.from_numpy(labels)

In [158]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batchnorm = nn.BatchNorm2d(3, affine=False)
        self.pad2 = nn.ConstantPad2d(2, 0)
        self.pad1 = nn.ConstantPad2d(1, 0)

        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 4, 5)
        self.conv3 = nn.Conv2d(4, 2, 5)
        self.conv4 = nn.Conv2d(2, 1, 3)

    def forward(self, x):
        x = self.batchnorm(x.float())
        x = self.pad1(F.relu(self.conv1(x)))
        x = self.pad2(F.relu(self.conv2(x)))
        x = self.pad2(F.relu(self.conv3(x)))
        x = self.pad1(F.relu(self.conv4(x)))

        x = x.view(-1, 225)
 
        return x

In [162]:
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
batchNum = 1000000
epochNum = 10
batchSize = 32
optimizer = optim.Adam(net.parameters(), lr=0.0001)
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [163]:
for epoch in range(epochNum):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(batchNum):
        # get the inputs
        inputs, labels = makeBatch(batchSize)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 5.249
[1,   400] loss: 4.783
[1,   600] loss: 4.655
[1,   800] loss: 4.616
[1,  1000] loss: 4.576
[1,  1200] loss: 4.547
[1,  1400] loss: 4.513
[1,  1600] loss: 4.481
[1,  1800] loss: 4.440
[1,  2000] loss: 4.367
[1,  2200] loss: 4.338
[1,  2400] loss: 4.325
[1,  2600] loss: 4.284
[1,  2800] loss: 4.233
[1,  3000] loss: 4.252
[1,  3200] loss: 4.191
[1,  3400] loss: 4.169
[1,  3600] loss: 4.153
[1,  3800] loss: 4.115
[1,  4000] loss: 4.101
[1,  4200] loss: 4.074
[1,  4400] loss: 4.068
[1,  4600] loss: 4.039
[1,  4800] loss: 3.997
[1,  5000] loss: 4.010
[1,  5200] loss: 4.000
[1,  5400] loss: 3.989
[1,  5600] loss: 3.958
[1,  5800] loss: 3.971
[1,  6000] loss: 3.961
[1,  6200] loss: 3.955
[1,  6400] loss: 3.924
[1,  6600] loss: 3.930
[1,  6800] loss: 3.916
[1,  7000] loss: 3.881
[1,  7200] loss: 3.884
[1,  7400] loss: 3.871
[1,  7600] loss: 3.850
[1,  7800] loss: 3.891
[1,  8000] loss: 3.849
[1,  8200] loss: 3.856
[1,  8400] loss: 3.817
[1,  8600] loss: 3.820
[1,  8800] 

[1, 71600] loss: 3.439
[1, 71800] loss: 3.439
[1, 72000] loss: 3.455
[1, 72200] loss: 3.418
[1, 72400] loss: 3.424
[1, 72600] loss: 3.402
[1, 72800] loss: 3.411
[1, 73000] loss: 3.419
[1, 73200] loss: 3.444
[1, 73400] loss: 3.386
[1, 73600] loss: 3.416
[1, 73800] loss: 3.401
[1, 74000] loss: 3.425
[1, 74200] loss: 3.424
[1, 74400] loss: 3.443
[1, 74600] loss: 3.417
[1, 74800] loss: 3.403
[1, 75000] loss: 3.381
[1, 75200] loss: 3.363
[1, 75400] loss: 3.434
[1, 75600] loss: 3.372
[1, 75800] loss: 3.410
[1, 76000] loss: 3.417
[1, 76200] loss: 3.400
[1, 76400] loss: 3.404
[1, 76600] loss: 3.406
[1, 76800] loss: 3.422
[1, 77000] loss: 3.399
[1, 77200] loss: 3.415
[1, 77400] loss: 3.412
[1, 77600] loss: 3.378
[1, 77800] loss: 3.425
[1, 78000] loss: 3.378
[1, 78200] loss: 3.387
[1, 78400] loss: 3.398
[1, 78600] loss: 3.408
[1, 78800] loss: 3.405
[1, 79000] loss: 3.396
[1, 79200] loss: 3.398
[1, 79400] loss: 3.387
[1, 79600] loss: 3.425
[1, 79800] loss: 3.389
[1, 80000] loss: 3.402
[1, 80200] 

[1, 141200] loss: 3.317
[1, 141400] loss: 3.353
[1, 141600] loss: 3.288
[1, 141800] loss: 3.359
[1, 142000] loss: 3.324
[1, 142200] loss: 3.277
[1, 142400] loss: 3.342
[1, 142600] loss: 3.323
[1, 142800] loss: 3.317
[1, 143000] loss: 3.322
[1, 143200] loss: 3.331
[1, 143400] loss: 3.291
[1, 143600] loss: 3.305
[1, 143800] loss: 3.314
[1, 144000] loss: 3.308
[1, 144200] loss: 3.309
[1, 144400] loss: 3.330
[1, 144600] loss: 3.322
[1, 144800] loss: 3.306
[1, 145000] loss: 3.296
[1, 145200] loss: 3.292
[1, 145400] loss: 3.309
[1, 145600] loss: 3.352
[1, 145800] loss: 3.305
[1, 146000] loss: 3.299
[1, 146200] loss: 3.309
[1, 146400] loss: 3.309
[1, 146600] loss: 3.316
[1, 146800] loss: 3.294
[1, 147000] loss: 3.303
[1, 147200] loss: 3.291
[1, 147400] loss: 3.331
[1, 147600] loss: 3.295
[1, 147800] loss: 3.291
[1, 148000] loss: 3.354
[1, 148200] loss: 3.323
[1, 148400] loss: 3.286
[1, 148600] loss: 3.333
[1, 148800] loss: 3.311
[1, 149000] loss: 3.324
[1, 149200] loss: 3.307
[1, 149400] loss

[1, 209600] loss: 3.204
[1, 209800] loss: 3.229
[1, 210000] loss: 3.256
[1, 210200] loss: 3.225
[1, 210400] loss: 3.257
[1, 210600] loss: 3.261
[1, 210800] loss: 3.290
[1, 211000] loss: 3.278
[1, 211200] loss: 3.253
[1, 211400] loss: 3.266
[1, 211600] loss: 3.262
[1, 211800] loss: 3.273
[1, 212000] loss: 3.273
[1, 212200] loss: 3.224
[1, 212400] loss: 3.238
[1, 212600] loss: 3.243
[1, 212800] loss: 3.265
[1, 213000] loss: 3.225
[1, 213200] loss: 3.263
[1, 213400] loss: 3.270
[1, 213600] loss: 3.237
[1, 213800] loss: 3.253
[1, 214000] loss: 3.244
[1, 214200] loss: 3.259
[1, 214400] loss: 3.239
[1, 214600] loss: 3.224
[1, 214800] loss: 3.254
[1, 215000] loss: 3.246
[1, 215200] loss: 3.284
[1, 215400] loss: 3.272
[1, 215600] loss: 3.244
[1, 215800] loss: 3.241
[1, 216000] loss: 3.246
[1, 216200] loss: 3.248
[1, 216400] loss: 3.233
[1, 216600] loss: 3.258
[1, 216800] loss: 3.251
[1, 217000] loss: 3.236
[1, 217200] loss: 3.237
[1, 217400] loss: 3.236
[1, 217600] loss: 3.224
[1, 217800] loss

[1, 278000] loss: 3.185
[1, 278200] loss: 3.198
[1, 278400] loss: 3.171
[1, 278600] loss: 3.194
[1, 278800] loss: 3.222
[1, 279000] loss: 3.192
[1, 279200] loss: 3.195
[1, 279400] loss: 3.190
[1, 279600] loss: 3.213
[1, 279800] loss: 3.207
[1, 280000] loss: 3.203
[1, 280200] loss: 3.216
[1, 280400] loss: 3.187
[1, 280600] loss: 3.186
[1, 280800] loss: 3.154
[1, 281000] loss: 3.195
[1, 281200] loss: 3.213
[1, 281400] loss: 3.209
[1, 281600] loss: 3.223
[1, 281800] loss: 3.190
[1, 282000] loss: 3.198
[1, 282200] loss: 3.211
[1, 282400] loss: 3.204
[1, 282600] loss: 3.220
[1, 282800] loss: 3.241
[1, 283000] loss: 3.195
[1, 283200] loss: 3.204
[1, 283400] loss: 3.211
[1, 283600] loss: 3.163
[1, 283800] loss: 3.210
[1, 284000] loss: 3.197
[1, 284200] loss: 3.195
[1, 284400] loss: 3.223
[1, 284600] loss: 3.195
[1, 284800] loss: 3.201
[1, 285000] loss: 3.193
[1, 285200] loss: 3.233
[1, 285400] loss: 3.217
[1, 285600] loss: 3.191
[1, 285800] loss: 3.210
[1, 286000] loss: 3.220
[1, 286200] loss

...

[5, 54600] loss: 3.051
[5, 54800] loss: 3.085
[5, 55000] loss: 3.022
[5, 55200] loss: 3.032
[5, 55400] loss: 3.024
[5, 55600] loss: 3.051
[5, 55800] loss: 3.036
[5, 56000] loss: 3.076
[5, 56200] loss: 3.065
[5, 56400] loss: 3.032
[5, 56600] loss: 3.039
[5, 56800] loss: 3.051
[5, 57000] loss: 2.984
[5, 57200] loss: 3.029
[5, 57400] loss: 3.054
[5, 57600] loss: 3.045
[5, 57800] loss: 3.051
[5, 58000] loss: 3.044
[5, 58200] loss: 3.061
[5, 58400] loss: 3.013
[5, 58600] loss: 3.080
[5, 58800] loss: 3.073
[5, 59000] loss: 3.011
[5, 59200] loss: 3.015
[5, 59400] loss: 3.026
[5, 59600] loss: 3.077
[5, 59800] loss: 3.074
[5, 60000] loss: 3.048
[5, 60200] loss: 3.066
[5, 60400] loss: 3.057
[5, 60600] loss: 3.060
[5, 60800] loss: 3.054
[5, 61000] loss: 3.041
[5, 61200] loss: 3.037
[5, 61400] loss: 3.043
[5, 61600] loss: 3.089
[5, 61800] loss: 3.018
[5, 62000] loss: 3.078
[5, 62200] loss: 3.022
[5, 62400] loss: 3.092
[5, 62600] loss: 3.064
[5, 62800] loss: 3.055
[5, 63000] loss: 3.057
[5, 63200] 

[5, 124800] loss: 3.105
[5, 125000] loss: 3.057
[5, 125200] loss: 3.093
[5, 125400] loss: 3.073
[5, 125600] loss: 3.046
[5, 125800] loss: 3.050
[5, 126000] loss: 3.074
[5, 126200] loss: 3.043
[5, 126400] loss: 3.068
[5, 126600] loss: 3.032
[5, 126800] loss: 3.053
[5, 127000] loss: 3.030
[5, 127200] loss: 3.067
[5, 127400] loss: 3.039
[5, 127600] loss: 3.049
[5, 127800] loss: 3.028
[5, 128000] loss: 3.041
[5, 128200] loss: 3.002
[5, 128400] loss: 3.045
[5, 128600] loss: 3.073
[5, 128800] loss: 3.016
[5, 129000] loss: 3.036
[5, 129200] loss: 3.061
[5, 129400] loss: 3.070
[5, 129600] loss: 3.046
[5, 129800] loss: 3.034
[5, 130000] loss: 3.098
[5, 130200] loss: 3.062
[5, 130400] loss: 3.047
[5, 130600] loss: 3.066
[5, 130800] loss: 3.048
[5, 131000] loss: 3.055
[5, 131200] loss: 3.052
[5, 131400] loss: 3.052
[5, 131600] loss: 3.057
[5, 131800] loss: 3.042
[5, 132000] loss: 3.045
[5, 132200] loss: 3.069
[5, 132400] loss: 3.072
[5, 132600] loss: 3.027
[5, 132800] loss: 3.058
[5, 133000] loss

[5, 193200] loss: 3.076
[5, 193400] loss: 3.050
[5, 193600] loss: 3.068
[5, 193800] loss: 3.020
[5, 194000] loss: 3.097
[5, 194200] loss: 3.030
[5, 194400] loss: 3.050
[5, 194600] loss: 3.079
[5, 194800] loss: 3.045
[5, 195000] loss: 3.073
[5, 195200] loss: 3.030
[5, 195400] loss: 3.039
[5, 195600] loss: 3.052
[5, 195800] loss: 3.071
[5, 196000] loss: 3.083
[5, 196200] loss: 3.081
[5, 196400] loss: 3.039
[5, 196600] loss: 3.084
[5, 196800] loss: 3.070
[5, 197000] loss: 3.038
[5, 197200] loss: 3.050
[5, 197400] loss: 3.088
[5, 197600] loss: 3.006
[5, 197800] loss: 3.066
[5, 198000] loss: 3.055
[5, 198200] loss: 3.089
[5, 198400] loss: 3.070
[5, 198600] loss: 3.038
[5, 198800] loss: 3.043
[5, 199000] loss: 3.012
[5, 199200] loss: 3.055
[5, 199400] loss: 3.034
[5, 199600] loss: 3.022
[5, 199800] loss: 3.074
[5, 200000] loss: 3.070
[5, 200200] loss: 3.066
[5, 200400] loss: 3.052
[5, 200600] loss: 3.051
[5, 200800] loss: 3.076
[5, 201000] loss: 3.051
[5, 201200] loss: 3.057
[5, 201400] loss

[5, 261600] loss: 3.031
[5, 261800] loss: 3.078
[5, 262000] loss: 3.052
[5, 262200] loss: 3.070
[5, 262400] loss: 3.024
[5, 262600] loss: 3.058
[5, 262800] loss: 3.069
[5, 263000] loss: 3.019
[5, 263200] loss: 3.020
[5, 263400] loss: 3.045
[5, 263600] loss: 3.008
[5, 263800] loss: 3.049
[5, 264000] loss: 3.050
[5, 264200] loss: 3.036
[5, 264400] loss: 3.057
[5, 264600] loss: 3.073
[5, 264800] loss: 3.050
[5, 265000] loss: 3.039
[5, 265200] loss: 3.041
[5, 265400] loss: 3.017
[5, 265600] loss: 3.080
[5, 265800] loss: 3.066
[5, 266000] loss: 3.023
[5, 266200] loss: 3.069
[5, 266400] loss: 3.082
[5, 266600] loss: 3.055
[5, 266800] loss: 3.036
[5, 267000] loss: 3.053
[5, 267200] loss: 3.008
[5, 267400] loss: 3.041
[5, 267600] loss: 3.068
[5, 267800] loss: 3.070
[5, 268000] loss: 3.076
[5, 268200] loss: 3.048
[5, 268400] loss: 3.066
[5, 268600] loss: 3.054
[5, 268800] loss: 3.050
[5, 269000] loss: 3.032
[5, 269200] loss: 3.039
[5, 269400] loss: 3.051
[5, 269600] loss: 3.073
[5, 269800] loss

KeyboardInterrupt: 

## Game

In [172]:
def toTripleBoard(board, side):
    curBoard = np.zeros(shape=(3, 15, 15), dtype=np.float)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.float)
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.float)

    if side == 1:
        blackTurnBoard = np.ones(shape=(15, 15), dtype=np.float)
    else:
        blackTurnBoard = -np.ones(shape=(15, 15), dtype=np.float)
    curBoard[0,:] = blackTurnBoard
    for i in range(15):
        for j in range(15):
            if board[i, j] == 1:
                blackBoard[i, j] = 1
            if board[i, j] == -1:
                whiteBoard[i, j] = -1
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    return curBoard


In [194]:
def printBoard(curBoard):
    print("-------------------------------------------------------------")
    cprint("     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 ", "blue")
    for i in range(15):
        if i < 10:
            cprint(" " + str(i), "blue", end=" ")
        else:
            cprint(i, "blue", end=" ")
        for j in range(15):
            if curBoard[i, j] == -1:
                print("|", end="")
                cprint(str(-1), 'red', "on_green", attrs=['blink'], end=" ")
            else:
                print("|", end = " ")
                if curBoard[i, j] == 1:
                    cprint(int(curBoard[i, j]), 'yellow', "on_blue", attrs=['blink'], end=" ")
                else:
                    print(int(curBoard[i, j]), end=" ")
        print("|")
        print("-------------------------------------------------------------")
    print()
    return


 ## (we are white)

In [195]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)
    print(netTurn)
    curBoard[netTurn // 15, netTurn % 15] = 1
    printBoard(curBoard)
    
    x, y = list(map(int, input().split()))
    curBoard[x, y] = -1
    #printBoard(curBoard)


tensor([128])
-------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
--------------------------------

9 6
tensor([113])
-------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 0 |-1 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
----------------------------

KeyboardInterrupt: 

## (we are black)

In [ ]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    x, y = list(map(int, input().split()))
    curBoard[x, y] = 1
    #printBoard(curBoard)
    
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)

    curBoard[netTurn // 15, netTurn % 15] = -1
    printBoard(curBoard)


5 5
-------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 1 |-1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
------------------------------------------

9 9
-------------------------------------------------------------
     0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |-1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 5 | 0 | 0 | 0 | 0 | 0 | 1 |-1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
 6 | 0 | 0 | 0 | 0 | 0 |-1 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
------------------------------------------

In [152]:

text = colored('Hello, World!', 'red', attrs=['reverse', 'blink'])
print(text)
cprint('Hello, World!', 'green', 'on_yellow', attrs=['blink'])

Hello, World!
Hello, World!
